In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
import time
import psutil
import csv
import os
import torchvision.models as models

In [2]:
# -------- Device Setup --------
if not torch.cuda.is_available():
    raise RuntimeError("CUDA is not available.")
device = torch.device("cuda")
print(f"Using GPU: {torch.cuda.get_device_name(0)}")

# -------- Dataset Setup --------
transform = transforms.Compose([
    transforms.Resize(224),  # ResNet expects 224x224
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
testset  = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True, num_workers=2)
testloader  = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False, num_workers=2)

# -------- Model Setup --------
model = models.resnet18(pretrained=False)
model.fc = nn.Linear(model.fc.in_features, 10)  # For CIFAR-10
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

Using GPU: Tesla T4


100%|██████████| 170M/170M [00:02<00:00, 75.7MB/s] 
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [3]:
# -------- CSV Logging Setup --------
csv_file = 'resnet18_pytorch_cuda.csv'
with open(csv_file, mode='w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow([
        "Epoch", "Image_Number", "Batch_Index", "Loss",
        "RAM_Usage_MB", "GPU_Alloc_MB", "Time_Per_Batch_s",
        "Train_Accuracy(%)", "Test_Accuracy(%)"
    ])

# -------- Training Loop --------
num_epochs = 10
model.train()

for epoch in range(num_epochs):
    correct_train, total_train = 0, 0

    for batch_idx, (images, labels) in enumerate(trainloader):
        start = time.perf_counter()

        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Accuracy tracking
        _, predicted = torch.max(outputs.data, 1)
        correct_train += (predicted == labels).sum().item()
        total_train += labels.size(0)

        end = time.perf_counter()

        # System metrics
        ram = psutil.Process(os.getpid()).memory_info().rss / (1024 * 1024)
        gpu_mem = torch.cuda.memory_allocated() / (1024 ** 2)
        elapsed = end - start
        image_number = batch_idx * images.size(0)

        # Log batch
        with open(csv_file, mode='a', newline='') as f:
            writer = csv.writer(f)
            writer.writerow([
                epoch + 1, image_number, batch_idx, f"{loss.item():.4f}",
                f"{ram:.2f}", f"{gpu_mem:.2f}", f"{elapsed:.6f}", "", ""
            ])

        if batch_idx % 100 == 0:
            print(f"[Epoch {epoch+1}] Image {image_number}, Batch {batch_idx}: "
                  f"Loss={loss.item():.4f}, RAM={ram:.2f}MB, GPU={gpu_mem:.2f}MB, Time={elapsed:.6f}s")

    # -------- Epoch Accuracy Logging --------
    train_accuracy = 100 * correct_train / total_train

    model.eval()
    correct_test, total_test = 0, 0
    with torch.no_grad():
        for images, labels in testloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            correct_test += (predicted == labels).sum().item()
            total_test += labels.size(0)
    test_accuracy = 100 * correct_test / total_test

    print(f"[Epoch {epoch+1}] Train Accuracy: {train_accuracy:.2f}% | Test Accuracy: {test_accuracy:.2f}%")
    model.train()

    # Log epoch summary
    with open(csv_file, mode='a', newline='') as f:
        writer = csv.writer(f)
        writer.writerow([
            epoch + 1, "END", "-", "-", "-", "-", "-",
            f"{train_accuracy:.2f}", f"{test_accuracy:.2f}"
        ])

print(f"Final GPU Memory Used: {torch.cuda.memory_allocated() / (1024 ** 2):.2f} MB")

[Epoch 1] Image 0, Batch 0: Loss=2.3525, RAM=1403.35MB, GPU=228.06MB, Time=1.248730s
[Epoch 1] Image 6400, Batch 100: Loss=1.6906, RAM=1404.37MB, GPU=225.68MB, Time=0.177925s
[Epoch 1] Image 12800, Batch 200: Loss=1.4454, RAM=1404.37MB, GPU=225.68MB, Time=0.182656s
[Epoch 1] Image 19200, Batch 300: Loss=1.3552, RAM=1404.44MB, GPU=225.68MB, Time=0.188235s
[Epoch 1] Image 25600, Batch 400: Loss=1.1507, RAM=1404.45MB, GPU=225.68MB, Time=0.195215s
[Epoch 1] Image 32000, Batch 500: Loss=1.2024, RAM=1404.62MB, GPU=225.68MB, Time=0.201311s
[Epoch 1] Image 38400, Batch 600: Loss=1.2549, RAM=1404.55MB, GPU=225.68MB, Time=0.217868s
[Epoch 1] Image 44800, Batch 700: Loss=1.0348, RAM=1404.51MB, GPU=225.68MB, Time=0.211615s
[Epoch 1] Train Accuracy: 51.79% | Test Accuracy: 59.72%
[Epoch 2] Image 0, Batch 0: Loss=1.0603, RAM=1433.47MB, GPU=225.56MB, Time=0.209345s
[Epoch 2] Image 6400, Batch 100: Loss=0.6612, RAM=1433.61MB, GPU=225.93MB, Time=0.214878s
[Epoch 2] Image 12800, Batch 200: Loss=0.9153, 